## Before training

This program saves the last 3 generations of models to Google Drive. Since 1 generation of models is >1GB, you should have at least 3GB of free space in Google Drive. If you do not have such free space, it is recommended to create another Google Account.

Training requires >10GB VRAM. (T4 should be enough) Inference does not require such a lot of VRAM.

## Installation

In [1]:
#@title Check GPU
!nvidia-smi

Tue Jun 24 13:41:59 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P0             28W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install -q gdown

In [3]:
!unzip -q songs.zip -d /kaggle/working/songs

unzip:  cannot find or open songs.zip, songs.zip.zip or songs.zip.ZIP.


In [4]:
#@title Install dependencies
#@markdown pip may fail to resolve dependencies and raise ERROR, but it can be ignored.
!python -m pip install -U pip wheel
%pip install -U ipython

#@markdown Branch (for development)

BRANCH = "none" #@param {"type": "string"}
if BRANCH == "none":
    %pip install -U so-vits-svc-fork
else:
    %pip install -U git+https://github.com/34j/so-vits-svc-fork.git@{BRANCH}

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.3/605.3 kB 8.0 MB/s eta 0:00:00
  Attempting uninstall: traitlets
    Found existing installation: traitlets 5.7.1
    Uninstalling traitlets-5.7.1:
      Successfully uninstalled traitlets-5.7.1
  Attempting uninstall: ipython
    Found existing installation: ipython 7.34.0
    Uninstalling ipython-7.34.0:
      Successfully uninstalled ipython-7.34.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [ipython]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.38.0, but you have google-auth 2.40.1 which is incompatible.
google-colab 1.0.0 requires ipython==7.3

## Training

In [5]:
F0_METHOD = "dio" #@param ["crepe", "crepe-tiny", "parselmouth", "dio", "harvest"]
!svc pre-hubert -fm {F0_METHOD}

2025-06-24 13:44:22.436379: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750772662.619262     139 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750772662.671406     139 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Usage: svc pre-hubert [OPTIONS]
Try 'svc pre-hubert -h' for help.

Error: Invalid value for '-i' / '--input-dir': Path 'dataset/44k' does not exist.


## Training Cluster model

## Inference

### Pretrained models

In [6]:
#@title Automatic preprocessing
!svc pre-resample

2025-06-24 13:44:47.653359: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750772687.675552     154 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750772687.682312     154 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Usage: svc pre-resample [OPTIONS]
Try 'svc pre-resample -h' for help.

Error: Invalid value for '-i' / '--input-dir': Path 'dataset_raw' does not exist.


In [7]:
!svc pre-config

2025-06-24 13:44:59.574294: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750772699.596261     169 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750772699.603035     169 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Usage: svc pre-config [OPTIONS]
Try 'svc pre-config -h' for help.

Error: Invalid value for '-i' / '--input-dir': Path 'dataset/44k' does not exist.


In [8]:
#@title Download pre-trained model with proper URLs
# Fix: Use 'resolve' instead of 'blob' for direct file download
!wget -N "https://huggingface.co/marcoc2/so-vits-svc-4.0-models/resolve/main/BillieJoe/G_24000.pth"
!wget -N "https://huggingface.co/marcoc2/so-vits-svc-4.0-models/resolve/main/BillieJoe/config.json"

import os
import shutil
import soundfile as sf



from IPython.display import Audio, display
from glob import glob
import subprocess
import json

# Configuration
input_folder = "/kaggle/input/directory1/Hindi"
temp_folder = "/kaggle/working/temp_audio"
output_folder = "/kaggle/working/processed-hindi"
config_path = "/kaggle/working/config.json"
model_path = "/kaggle/working/G_24000.pth"

os.makedirs(temp_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

# Validate downloaded files before processing
print("Validating downloaded files...")

# Check if model file exists and has reasonable size
if os.path.exists(model_path):
    model_size = os.path.getsize(model_path)
    print(f"✓ Model file found: {model_path} ({model_size / (1024*1024):.1f} MB)")
    if model_size < 1024:  # Less than 1KB indicates download failure
        print("❌ Model file seems too small - redownload required")
        exit(1)
else:
    print(f"❌ Model file not found: {model_path}")
    exit(1)

# Check if config file exists and is valid JSON
if os.path.exists(config_path):
    config_size = os.path.getsize(config_path)
    print(f"✓ Config file found: {config_path} ({config_size} bytes)")

    # Validate JSON content
    try:
        with open(config_path, 'r', encoding='utf-8') as f:
            config_content = f.read().strip()
            if not config_content:
                print("❌ Config file is empty")
                exit(1)
            config_data = json.loads(config_content)
            print("✓ Config file is valid JSON")
    except json.JSONDecodeError as e:
        print(f"❌ Config file contains invalid JSON: {e}")
        print("Config file content preview:")
        with open(config_path, 'r', encoding='utf-8') as f:
            print(repr(f.read()[:200]))
        exit(1)
    except Exception as e:
        print(f"❌ Error reading config file: {e}")
        exit(1)
else:
    print(f"❌ Config file not found: {config_path}")
    exit(1)

print("✓ All files validated successfully!\n")

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Find all audio files (support multiple formats)
audio_extensions = ["*.wav", "*.mp3", "*.flac", "*.m4a", "*.ogg"]
audio_files = []
for ext in audio_extensions:
    audio_files.extend(glob(os.path.join(input_folder, ext)))

for file in audio_files:
    shutil.copy(file, temp_folder)

print(f"✅ Copied {len(audio_files)} files to temp folder for processing.")

# Processing
temp_audio_files = sorted(glob(os.path.join(temp_folder, "*")))

success = 0
fail = 0

for i, audio_path in enumerate(temp_audio_files, 1):
    filename = os.path.basename(audio_path)
    name, ext = os.path.splitext(filename)
    print(f"\n[{i}/{len(temp_audio_files)}] 🔄 Processing: {filename}")

    # Define explicit output path
    out_path = os.path.join(output_folder, f"{name}_processed.wav")

    try:
        result = subprocess.run([
            "svc", "infer", audio_path,
            "-c", config_path,
            "-m", model_path,
            "--output-path", out_path
        ], capture_output=True, text=True, timeout=300)

        if result.returncode != 0:
            print(f"✗ SVC failed: {result.stderr}")
            fail += 1
            continue

        if os.path.exists(out_path):
            print(f"✅ Output saved: {out_path}")
            success += 1
        else:
            print("⚠ Output file not found.")
            fail += 1

    except subprocess.TimeoutExpired:
        print("⏰ Timeout!")
        fail += 1
    except Exception as e:
        print(f"❌ Error: {e}")
        fail += 1

print(f"\n🎉 Processing complete!")
print(f"✓ Successfully processed: {successful_count} files")
print(f"✗ Failed to process: {failed_count} files")
print(f"📁 Output folder: {output_folder}")


--2025-06-24 13:45:05--  https://huggingface.co/marcoc2/so-vits-svc-4.0-models/resolve/main/BillieJoe/G_24000.pth
Resolving huggingface.co (huggingface.co)... 3.169.137.5, 3.169.137.119, 3.169.137.19, ...
Connecting to huggingface.co (huggingface.co)|3.169.137.5|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/5a/37/5a37557be79c3c503e096c13cea087ab168030695da8ae0921856b6cb4d7d553/0448ac2562aeccf65c2d08bde5ac3b9867c83da33c470da5033553d5890d5b99?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27G_24000.pth%3B+filename%3D%22G_24000.pth%22%3B&Expires=1750776305&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1MDc3NjMwNX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy81YS8zNy81YTM3NTU3YmU3OWMzYzUwM2UwOTZjMTNjZWEwODdhYjE2ODAzMDY5NWRhOGFlMDkyMTg1NmI2Y2I0ZDdkNTUzLzA0NDhhYzI1NjJhZWNjZjY1YzJkMDhiZGU1YWMzYjk4NjdjODNkYTMzYzQ3MGRhNTAzMzU1M2Q1ODkwZDViOTk%7EcmVzcG9uc2UtY29udGVudC1ka

NameError: name 'successful_count' is not defined

In [ ]:
import shutil

# Zip the output folder
shutil.make_archive("/kaggle/working/processed-hindi", 'zip', "/kaggle/working/processed-hindi")
